In [1]:
from rich.console import Console
from rich.panel import Panel
from rich.text import Text
import json
console = Console()

In [ ]:
import os
os.environ["TAVILY_API_KEY"]='add api key here'
os.environ['GOOGLE_API_KEY']="add api key here"

from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.083,   # ~5 requests per minute
    check_every_n_seconds=0.1,   # wake up every 100 ms to check
    max_bucket_size=1            # allow only 1 request at a time (no bursts)
)


model=init_chat_model("gemini-2.5-pro",model_provider="google_genai",temperature=0,rate_limiter=rate_limiter)